In [ ]:
root = "../"
%store -r root

In [ ]:
from yaml import SafeLoader, load
from os import path

with open(path.join(root, "inventory", "keys.yml"), "r") as key_file:
  keys = load(key_file, Loader=SafeLoader)

vars = {}

def findKey(name: str) -> str:
  return keys["keys"][name]

for name, members in keys["groups"].items():
  group_keys = map(findKey, members)
  vars[name] = ','.join(group_keys)

%store vars

In [ ]:
from jinja2 import Environment
from os import path
from yaml import SafeLoader, load

%store -r root
%store -r vars

with open(path.join(root, ".sops.yaml.j2"), "r") as template_file:
  sops_template = template_file.read()

environment = Environment()
template = environment.from_string(sops_template)
sops_config_text = template.render(vars)

sops_config = load(sops_config_text, Loader=SafeLoader)
%store sops_config

sops_config

In [ ]:
from yaml import dump
from os import path

%store -r root
%store -r sops_config

with open(path.join(root, ".sops.yaml"), "w") as sops_file:
  dump(sops_config, sops_file)